# PUTATIVE WORKFLOW


## WORKFLOW EDITOR PLUGIN
- FINE-TUNE SEGMENTATIONS
  - export workflow.jsons
    - masks:
      - nuclei
      - cellmask
      - cytoplasm
    - organelles:
      - lyso
      - mito
      - golgi
      - perox
      - ER
      - LD


## BATCHPROCESS WORKFLOW
- BATCH PROCESS
  - load workflow.jsons for: 
  1. masks
    - export: masks .tiff as stack (nuclei, cellmask, cytoplasm)
  2. organelles
    - export individual .tiffs



## NOTEBOOK ~~OR ***FUTURE*** PLUGIN~~
- COLLECT ORGANELLE STATS
  - extract masks.tiffs as individual
    - nuclei, cellmask, cytoplasm
  - collect regionprops for all organelles
    - export .csvs


## NOTEBOOK ~~OR __FUTURE__ PLUGIN~~
- SUMMARIZE STUDY DATA
  - munge .csv to create summary stats across all cells/images




_____________

## TO DO
- add "segmentation name" field instead of copying from workflow.json name


- choose alternate conf_XXX.json location. 
  - strategy:  add to "prebuilt" list from path


  
  ## FILE NAME CONVENTIONS

  raw file name is kept.

  PREFIX = "segmentation name" or regionprop name.  e.g. 
  SUFFIX = "description" i.e. 

In [5]:
# top level imports
from pathlib import Path
import os, sys
from typing import Optional, Union, Dict, List

import numpy as np
import pandas as pd

import napari

### import local python functions in ../infer_subc
sys.path.append(os.path.abspath((os.path.join(os.getcwd(), '..'))))

from infer_subc.core.file_io import (read_czi_image,
                                        export_inferred_organelle,
                                        import_inferred_organelle,
                                        export_tiff,
                                        list_image_files)



import infer_subc.constants

import time
%load_ext autoreload
%autoreload 2



In [2]:
# this will be the example image for testing the pipeline below
# build the datapath
# all the imaging data goes here.
data_root_path = Path(os.path.expanduser("~")) / "Projects/Imaging/data"

# linearly unmixed ".czi" files are here
int_data_path = data_root_path / "intermediate"
im_type = ".tiff"

# get the list of all files
img_file_list = list_image_files(int_data_path,im_type)

# save output ".tiff" files here
out_data_path = data_root_path / "out"

if not Path.exists(out_data_path):
    Path.mkdir(out_data_path)
    print(f"making {out_data_path}")

In [3]:
im_path = Path(img_file_list[0])
im_path

PosixPath('/Users/ergonyc/Projects/Imaging/data/intermediate/ZSTACK_PBTOhNGN2hiPSCs_BR1_N16_Unmixed-ER.tiff')

## 1. get each unique cells accouding to filename


### extract ID. e.g.

### process each cell & summarize



## 2. aggregate all cells into a database 

In [8]:
full_name = im_path.name

cell_ids = [ Path(fn).stem.split("-")[0] for fn in img_file_list]
cell_ids = list(set(cell_ids))

masks_postfix = "masks2"
organelle_postfix = ["lyso", "mito","golgi","perox","ER","LD"]




# function to get masks, and organeles

def gather_segmentations(file_id:str, path_root: Path, masks_postfix: str, organelle_postfix: List[str] ) -> List[np.ndarray]:
    """
    

    """
    pass


    # create filename ID 


    # 

{'ZSTACK_PBTOhNGN2hiPSCs_BR1_N20_Unmixed',
 'ZSTACK_PBTOhNGN2hiPSCs_BR1_N21_Unmixed',
 'ZSTACK_PBTOhNGN2hiPSCs_BR1_N22_Unmixed',
 'ZSTACK_PBTOhNGN2hiPSCs_BR1_N23_Unmixed',
 'ZSTACK_PBTOhNGN2hiPSCs_BR2_N01_Unmixed',
 'ZSTACK_PBTOhNGN2hiPSCs_BR2_N02_Unmixed',
 'ZSTACK_PBTOhNGN2hiPSCs_BR2_N03_Unmixed',
 'ZSTACK_PBTOhNGN2hiPSCs_BR2_N04_Unmixed',
 'ZSTACK_PBTOhNGN2hiPSCs_BR2_N05_Unmixed',
 'ZSTACK_PBTOhNGN2hiPSCs_BR2_N06_Unmixed',
 'ZSTACK_PBTOhNGN2hiPSCs_BR2_N07_Unmixed',
 'ZSTACK_PBTOhNGN2hiPSCs_BR2_N08_Unmixed',
 'ZSTACK_PBTOhNGN2hiPSCs_BR2_N09_Unmixed',
 'ZSTACK_PBTOhNGN2hiPSCs_BR2_N10_Unmixed',
 'ZSTACK_PBTOhNGN2hiPSCs_BR3_N01_Unmixed',
 'ZSTACK_PBTOhNGN2hiPSCs_BR3_N02_Unmixed',
 'ZSTACK_PBTOhNGN2hiPSCs_BR3_N04_Unmixed',
 'ZSTACK_PBTOhNGN2hiPSCs_BR3_N05_Unmixed',
 'ZSTACK_PBTOhNGN2hiPSCs_BR3_N06_Unmixed',
 'ZSTACK_PBTOhNGN2hiPSCs_BR3_N07_Unmixed',
 'ZSTACK_PBTOhNGN2hiPSCs_BR3_N08_Unmixed',
 'ZSTACK_PBTOhNGN2hiPSCs_BR3_N09_Unmixed',
 'ZSTACK_PBTOhNGN2hiPSCs_BR3_N10_Unmixed',
 'ZSTACK_PB

In [63]:
# MASK process
# 1. get a listof all files based on a "prefix" and "suffix" for a given path
# dump three .tiff from teh mask multichannel tiff
# from tifffile import imwrite, imread#, tiffcomment
from infer_subc.core.img import label_uint16
from infer_subc.core.file_io import export_tiff, read_tiff_image
from typing import Union
from pathlib import Path

def explode_mask(mask_path: Union[Path,str], postfix: str= "masks", im_type: str = ".tiff") -> bool:
    """ 
    TODO: add loggin instead of printing
        append tiffcomments with provenance
    """
    if isinstance(mask_path, str): mask_path = Path(mask_path)
    # load image 
    full_stem = mask_path.stem
    if full_stem.endswith(postfix):
        stem = full_stem.rstrip(postfix)
        image = read_tiff_image(mask_path)
        assert image.shape[0]==3
        
        # make into np.uint16 labels
        nuclei = label_uint16(image[0])
        # export as np.uint8 (255)
        cellmask = image[1]>0            
        cytosol = image[2]>0

        # write wasks
        root_stem = mask_path.parent / stem
        # ret1 = imwrite(f"{root}nuclei{stem}", nuclei)
        ret1 = export_tiff(nuclei, f"{stem}nuclei", mask_path.parent, None)
        # ret2 = imwrite(f"{root}cellmask{stem}", cellmask)
        ret2 = export_tiff(cellmask, f"{stem}cellmask", mask_path.parent, None)
        # ret3 = imwrite(f"{root}cytosol{stem}", cytosol)
        ret3 = export_tiff(cytosol, f"{stem}cyto", mask_path.parent, None)

        print(f"wrote {stem}-{{nuclei,cellmask,cytosol}}")
        return True
    else:
        return False



def explode_masks(root_path: Union[Path,str], postfix: str= "masks", im_type: str = ".tiff"):
    """  
    TODO: add loggin instead of printing
        append tiffcomments with provenance
    """
    if isinstance(root_path, str): root_path = Path(root_path)
    img_file_list = list_image_files(root_path,im_type, postfix)
    wrote_cnt = 0
    for img_f in img_file_list:
        if explode_mask(img_f, postfix=postfix, im_type=im_type): wrote_cnt += 1
        else: print(f"failed to explode {img_f}")
    else:
        print(f"how thefark!!! {img_f}")

    print(f"exploded {wrote_cnt*100./len(img_file_list)} pct of {len(img_file_list)} files")
    return wrote_cnt



In [64]:
cnt = explode_masks(out_data_path)
cnt

>>>>>>>>>>>> tifffile.imread  (dtype=uint8 in (0.01) sec
export dtype - {dtype}
>>>>>>>>>>>> tifffile.imwrite in (0.01) sec
changed dtype from bool to uint8
>>>>>>>>>>>> tifffile.imwrite in (0.01) sec
changed dtype from bool to uint8
>>>>>>>>>>>> tifffile.imwrite in (0.01) sec
wrote ZSTACK_PBTOhNGN2hiPSCs_BR1_N14_Unmixed--{nuclei,cellmask,cytosol}
>>>>>>>>>>>> tifffile.imread  (dtype=uint8 in (0.01) sec
export dtype - {dtype}
>>>>>>>>>>>> tifffile.imwrite in (0.01) sec
changed dtype from bool to uint8
>>>>>>>>>>>> tifffile.imwrite in (0.02) sec
changed dtype from bool to uint8
>>>>>>>>>>>> tifffile.imwrite in (0.01) sec
wrote ZSTACK_PBTOhNGN2hiPSCs_BR1_N15_Unmixed--{nuclei,cellmask,cytosol}
>>>>>>>>>>>> tifffile.imread  (dtype=uint8 in (0.01) sec
export dtype - {dtype}
>>>>>>>>>>>> tifffile.imwrite in (0.01) sec
changed dtype from bool to uint8
>>>>>>>>>>>> tifffile.imwrite in (0.01) sec
changed dtype from bool to uint8
>>>>>>>>>>>> tifffile.imwrite in (0.01) sec
wrote ZSTACK_PBTOhNGN2

34

In [46]:


# all the imaging data goes here.
data_root_path = Path(os.path.expanduser("~")) / "Projects/Imaging/data"
# linearly unmixed ".czi" files are here
raw_data_path = data_root_path / "raw"
# save output ".tiff" files here
int_data_path = data_root_path / "intermediate"
# save stats here
out_data_path = data_root_path / "out"


[PosixPath('/Users/ahenrie/Projects/Imaging/data/raw/ZSTACK_PBTOhNGN2hiPSCs_BR1_N14_Unmixed.czi'),
 PosixPath('/Users/ahenrie/Projects/Imaging/data/raw/ZSTACK_PBTOhNGN2hiPSCs_BR1_N15_Unmixed.czi'),
 PosixPath('/Users/ahenrie/Projects/Imaging/data/raw/ZSTACK_PBTOhNGN2hiPSCs_BR1_N16_Unmixed.czi'),
 PosixPath('/Users/ahenrie/Projects/Imaging/data/raw/ZSTACK_PBTOhNGN2hiPSCs_BR1_N17_Unmixed.czi'),
 PosixPath('/Users/ahenrie/Projects/Imaging/data/raw/ZSTACK_PBTOhNGN2hiPSCs_BR1_N18_Unmixed.czi'),
 PosixPath('/Users/ahenrie/Projects/Imaging/data/raw/ZSTACK_PBTOhNGN2hiPSCs_BR1_N19_Unmixed.czi'),
 PosixPath('/Users/ahenrie/Projects/Imaging/data/raw/ZSTACK_PBTOhNGN2hiPSCs_BR1_N20_Unmixed.czi'),
 PosixPath('/Users/ahenrie/Projects/Imaging/data/raw/ZSTACK_PBTOhNGN2hiPSCs_BR1_N21_Unmixed.czi'),
 PosixPath('/Users/ahenrie/Projects/Imaging/data/raw/ZSTACK_PBTOhNGN2hiPSCs_BR1_N22_Unmixed.czi'),
 PosixPath('/Users/ahenrie/Projects/Imaging/data/raw/ZSTACK_PBTOhNGN2hiPSCs_BR1_N23_Unmixed.czi'),
 PosixPath

In [48]:
raw_path = raw_data_path
int_path = int_data_path
out_path = out_data_path


if isinstance(raw_path, str): raw_path = Path(raw_path)
if isinstance(int_path, str): int_path = Path(int_path)
if isinstance(out_path, str): out_path = Path(out_path)

img_file_list = list_image_files(raw_path,".czi")

if not Path.exists(out_path):
    Path.mkdir(out_path)
    print(f"making {out_path}")



True

In [68]:
from typing import Optional, Union, Dict, List

def find_segmentation_tiff_files(prototype:Union[Path,str], organelles: List[str], int_path: Union[Path,str]) -> Dict:
    """
    find the nescessary image files based on protype, the organelles involved, and paths
    """

    # raw
    prototype = Path(prototype)
    if not prototype.exists():
        print(f"bad prototype. please choose an existing `raw` file as prototype")
        return dict()
    # make sure protoype ends with czi

    out_files = {"raw":prototype}

    int_path = Path(int_path) 
    # raw
    if not int_path.is_dir():
        print(f"bad path argument. please choose an existing path containing organelle segmentations")
        return out_files
    
    # cyto, cellmask
    cyto_nm = int_path / f"{prototype.stem}-cyto.tiff"
    if cyto_nm.exists():
        out_files["cyto"] = cyto_nm
    else:
        print(f"cytosol mask not found.  We'll try to extract from masks ")
        if explode_mask(int_path / f"{prototype.stem}-masks.tiff"): 
            out_files["cyto"] = cyto_nm
        else: 
            print(f"failed to explode {prototype.stem}-masks.tiff")
            return out_files
    
    cellmask_nm = int_path / f"{prototype.stem}-cellmask.tiff"
    if  cellmask_nm.exists():
        out_files["cellmask"] = cellmask_nm
    else:
        print(f"cellmask file not found in {int_path} returning")
        out_files["cellmask"] = None

    # organelles
    for org_n in organelles:
        org_name = Path(int_path) / f"{prototype.stem}-{org_n}.tiff"
        if org_name.exists(): 
            out_files[org_n] = org_name
        else: 
            print(f"{org_n} .tiff file not found in {int_path} returning")
            out_files[org_n] = None
    
    return out_files

    


In [69]:
prototype = '/Users/ahenrie/Projects/Imaging/data/raw/ZSTACK_PBTOhNGN2hiPSCs_BR1_N14_Unmixed.czi'
organelles = ["nuclei","golgi","perox"]

find_segmentation_tiff_files(prototype, organelles, out_data_path)


{'raw': PosixPath('/Users/ahenrie/Projects/Imaging/data/raw/ZSTACK_PBTOhNGN2hiPSCs_BR1_N14_Unmixed.czi'),
 'cyto': PosixPath('/Users/ahenrie/Projects/Imaging/data/out/ZSTACK_PBTOhNGN2hiPSCs_BR1_N14_Unmixed-cyto.tiff'),
 'cellmask': PosixPath('/Users/ahenrie/Projects/Imaging/data/out/ZSTACK_PBTOhNGN2hiPSCs_BR1_N14_Unmixed-cellmask.tiff'),
 'nuclei': PosixPath('/Users/ahenrie/Projects/Imaging/data/out/ZSTACK_PBTOhNGN2hiPSCs_BR1_N14_Unmixed-nuclei.tiff'),
 'golgi': PosixPath('/Users/ahenrie/Projects/Imaging/data/out/ZSTACK_PBTOhNGN2hiPSCs_BR1_N14_Unmixed-golgi.tiff'),
 'perox': PosixPath('/Users/ahenrie/Projects/Imaging/data/out/ZSTACK_PBTOhNGN2hiPSCs_BR1_N14_Unmixed-perox.tiff')}

In [81]:
from infer_subc.utils.stats_helpers import shell_cross_stats, organelle_stats
from infer_subc.constants import *
from infer_subc.organelles import *
from infer_subc.core.file_io import read_tiff_image, read_czi_image

# names of organelles we have
organelle_names = ["nuclei","lyso", "mito","golgi","perox","ER","LD"]

get_methods  = [get_lyso,
            get_mito,
            get_golgi,
            get_perox,
            get_ER,
            get_LD]

# load all the organelle segmentations
# organelles = [meth(img_data,meta_dict, out_data_path) for meth in get_methods]

# get the intensities
organelle_channels = [NUC_CH, LYSO_CH,MITO_CH,GOLGI_CH,PEROX_CH,ER_CH,LD_CH]



In [92]:
# for a list of "prefixes"  collect stats + cross stats masked by cytosol (including nuclei masked by cellmask)

def load_cross_stats(int_path: Union[Path,str], out_path: Union[Path, str], raw_path: Union[Path,str], organelle_names: List[str]= ["nuclei","golgi","peroxi"], organelle_chs: List[int]= [NUC_CH,GOLGI_CH, PEROX_CH], postfix: str = ".tiff"):
    """  
    TODO: add loggin instead of printing
        append tiffcomments with provenance
    """

    
    if isinstance(raw_path, str): raw_path = Path(raw_path)
    if isinstance(int_path, str): int_path = Path(int_path)
    if isinstance(out_path, str): out_path = Path(out_path)
    
    img_file_list = list_image_files(raw_path,".czi")

    if not Path.exists(out_path):
        Path.mkdir(out_path)
        print(f"making {out_path}")
        
    print(img_file_list)
    for img_f in img_file_list:
        filez = find_segmentation_tiff_files(img_f, organelle_names, int_path)
        img_data,meta_dict = read_czi_image(filez['raw'])

        # load organelles and masks
        cyto = read_tiff_image(filez['cyto'])
        cellmask = read_tiff_image(filez['cellmask'])
        # create intensities from raw as list
        intensities = [img_data[ch] for ch in organelle_chs]

        # load organelles as list
        organelles = [ read_tiff_image(filez[org])for org in organelle_names]
        
        #get mask (cyto_mask)

        n_files = organelle_stats(organelle_names, organelles,intensities, cyto, out_path, img_f)
        print(n_files)

        n_files = shell_cross_stats(organelle_names, organelles, cyto, out_path, img_f) 




# all the imaging data goes here.
data_root_path = Path(os.path.expanduser("~")) / "Projects/Imaging/data"
# linearly unmixed ".czi" files are here
raw_data_path = data_root_path / "raw"
# save output ".tiff" files here
int_data_path = data_root_path / "intermediate"
# save stats here
out_data_path = data_root_path / "out"

get_cross_stats(out_data_path, out_data_path, raw_data_path, organelle_names=organelle_names, organelle_chs= organelle_channels)



[PosixPath('/Users/ahenrie/Projects/Imaging/data/raw/ZSTACK_PBTOhNGN2hiPSCs_BR1_N14_Unmixed.czi'), PosixPath('/Users/ahenrie/Projects/Imaging/data/raw/ZSTACK_PBTOhNGN2hiPSCs_BR1_N15_Unmixed.czi'), PosixPath('/Users/ahenrie/Projects/Imaging/data/raw/ZSTACK_PBTOhNGN2hiPSCs_BR1_N16_Unmixed.czi'), PosixPath('/Users/ahenrie/Projects/Imaging/data/raw/ZSTACK_PBTOhNGN2hiPSCs_BR1_N17_Unmixed.czi'), PosixPath('/Users/ahenrie/Projects/Imaging/data/raw/ZSTACK_PBTOhNGN2hiPSCs_BR1_N18_Unmixed.czi'), PosixPath('/Users/ahenrie/Projects/Imaging/data/raw/ZSTACK_PBTOhNGN2hiPSCs_BR1_N19_Unmixed.czi'), PosixPath('/Users/ahenrie/Projects/Imaging/data/raw/ZSTACK_PBTOhNGN2hiPSCs_BR1_N20_Unmixed.czi'), PosixPath('/Users/ahenrie/Projects/Imaging/data/raw/ZSTACK_PBTOhNGN2hiPSCs_BR1_N21_Unmixed.czi'), PosixPath('/Users/ahenrie/Projects/Imaging/data/raw/ZSTACK_PBTOhNGN2hiPSCs_BR1_N22_Unmixed.czi'), PosixPath('/Users/ahenrie/Projects/Imaging/data/raw/ZSTACK_PBTOhNGN2hiPSCs_BR1_N23_Unmixed.czi'), PosixPath('/Users/a

/opt/anaconda3/envs/cento/lib/python3.9/site-packages/ome_types/_convenience.py:106: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  d = to_dict(os.fspath(xml), parser=parser, validate=validate)


>>>>>>>>>>>> tifffile.imread  (dtype=uint8 in (0.01) sec
>>>>>>>>>>>> tifffile.imread  (dtype=uint8 in (0.00) sec
>>>>>>>>>>>> tifffile.imread  (dtype=uint16 in (0.01) sec
>>>>>>>>>>>> tifffile.imread  (dtype=uint16 in (0.02) sec
>>>>>>>>>>>> tifffile.imread  (dtype=uint16 in (0.01) sec
>>>>>>>>>>>> tifffile.imread  (dtype=uint16 in (0.01) sec
>>>>>>>>>>>> tifffile.imread  (dtype=uint16 in (0.02) sec
>>>>>>>>>>>> tifffile.imread  (dtype=uint16 in (0.03) sec
>>>>>>>>>>>> tifffile.imread  (dtype=uint16 in (0.02) sec
getting stats for A = nuclei
  b = lyso
  b = mito
  b = golgi
  b = perox
  b = ER
  b = LD
getting stats for A = lyso
  b = nuclei
  b = mito
  b = golgi
  b = perox
  b = ER
  b = LD
getting stats for A = mito
  b = nuclei
  b = lyso
  b = golgi
  b = perox
  b = ER
  b = LD
getting stats for A = golgi
  b = nuclei
  b = lyso
  b = mito
  b = perox
  b = ER
  b = LD
getting stats for A = perox
  b = nuclei
  b = lyso
  b = mito
  b = golgi
  b = ER
  b = LD
getting stats f

/opt/anaconda3/envs/cento/lib/python3.9/site-packages/ome_types/_convenience.py:106: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  d = to_dict(os.fspath(xml), parser=parser, validate=validate)


>>>>>>>>>>>> tifffile.imread  (dtype=uint8 in (0.01) sec
>>>>>>>>>>>> tifffile.imread  (dtype=uint8 in (0.00) sec
>>>>>>>>>>>> tifffile.imread  (dtype=uint16 in (0.01) sec
>>>>>>>>>>>> tifffile.imread  (dtype=uint16 in (0.02) sec
>>>>>>>>>>>> tifffile.imread  (dtype=uint16 in (0.01) sec
>>>>>>>>>>>> tifffile.imread  (dtype=uint16 in (0.03) sec
>>>>>>>>>>>> tifffile.imread  (dtype=uint16 in (0.02) sec
>>>>>>>>>>>> tifffile.imread  (dtype=uint16 in (0.02) sec
>>>>>>>>>>>> tifffile.imread  (dtype=uint16 in (0.01) sec
getting stats for A = nuclei
  b = lyso
  b = mito
  b = golgi
  b = perox
  b = ER
  b = LD
getting stats for A = lyso
  b = nuclei
  b = mito
  b = golgi
  b = perox
  b = ER
  b = LD
getting stats for A = mito
  b = nuclei
  b = lyso
  b = golgi
  b = perox
  b = ER
  b = LD
getting stats for A = golgi
  b = nuclei
  b = lyso
  b = mito
  b = perox
  b = ER
  b = LD
getting stats for A = perox
  b = nuclei
  b = lyso
  b = mito
  b = golgi
  b = ER
  b = LD
getting stats f

...

/opt/anaconda3/envs/cento/lib/python3.9/site-packages/ome_types/_convenience.py:106: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  d = to_dict(os.fspath(xml), parser=parser, validate=validate)


>>>>>>>>>>>> tifffile.imread  (dtype=uint8 in (0.01) sec
>>>>>>>>>>>> tifffile.imread  (dtype=uint8 in (0.01) sec
>>>>>>>>>>>> tifffile.imread  (dtype=uint16 in (0.01) sec
>>>>>>>>>>>> tifffile.imread  (dtype=uint16 in (0.02) sec
>>>>>>>>>>>> tifffile.imread  (dtype=uint16 in (0.02) sec
>>>>>>>>>>>> tifffile.imread  (dtype=uint16 in (0.02) sec
>>>>>>>>>>>> tifffile.imread  (dtype=uint16 in (0.03) sec
>>>>>>>>>>>> tifffile.imread  (dtype=uint16 in (0.04) sec
>>>>>>>>>>>> tifffile.imread  (dtype=uint16 in (0.03) sec
getting stats for A = nuclei
  b = lyso
  b = mito
  b = golgi
  b = perox
  b = ER
  b = LD
getting stats for A = lyso
  b = nuclei
  b = mito
  b = golgi
  b = perox
  b = ER
  b = LD
getting stats for A = mito
  b = nuclei
  b = lyso
  b = golgi
  b = perox
  b = ER
  b = LD
getting stats for A = golgi
  b = nuclei
  b = lyso
  b = mito
  b = perox
  b = ER
  b = LD
getting stats for A = perox
  b = nuclei
  b = lyso
  b = mito
  b = golgi
  b = ER
  b = LD
getting stats f

/opt/anaconda3/envs/cento/lib/python3.9/site-packages/ome_types/_convenience.py:106: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  d = to_dict(os.fspath(xml), parser=parser, validate=validate)


>>>>>>>>>>>> tifffile.imread  (dtype=uint8 in (0.01) sec
>>>>>>>>>>>> tifffile.imread  (dtype=uint8 in (0.02) sec
>>>>>>>>>>>> tifffile.imread  (dtype=uint16 in (0.03) sec
>>>>>>>>>>>> tifffile.imread  (dtype=uint16 in (0.03) sec
>>>>>>>>>>>> tifffile.imread  (dtype=uint16 in (0.03) sec
>>>>>>>>>>>> tifffile.imread  (dtype=uint16 in (0.03) sec
>>>>>>>>>>>> tifffile.imread  (dtype=uint16 in (0.03) sec
>>>>>>>>>>>> tifffile.imread  (dtype=uint16 in (0.03) sec
>>>>>>>>>>>> tifffile.imread  (dtype=uint16 in (0.06) sec
getting stats for A = nuclei
  b = lyso
  b = mito
  b = golgi
  b = perox
  b = ER
  b = LD
getting stats for A = lyso
  b = nuclei
  b = mito
  b = golgi
  b = perox
  b = ER
  b = LD
getting stats for A = mito
  b = nuclei
  b = lyso
  b = golgi
  b = perox
  b = ER
  b = LD
getting stats for A = golgi
  b = nuclei
  b = lyso
  b = mito
  b = perox
  b = ER
  b = LD
getting stats for A = perox
  b = nuclei
  b = lyso
  b = mito
  b = golgi
  b = ER
  b = LD
getting stats f

FileNotFoundError: [Errno 2] No such file or directory: '/Users/ahenrie/Projects/Imaging/mcz_subcell/data/out/ZSTACK_PBTOhNGN2hiPSCs_BR3_N04_Unmixed-masks.tiff'

In [66]:
organelles = ["nuclei","golgi","peroxi"]

organelles.count("xxxx")

0

# summary statistics

We now need to merge our files

In [19]:

data_root_path = Path(os.path.expanduser("~")) / "Projects/Imaging/data"

# linearly unmixed ".czi" files are here
int_data_path = data_root_path / "out"
im_type = ".csv"

# get the list of all files
csv_file_list = list_image_files(int_data_path,im_type)

In [14]:
csv_file_list[1].name

'ZSTACK_PBTOhNGN2hiPSCs_BR1_N19_Unmixed-cytosol-stats.csv'

In [ ]:
pd.melt(df, id_vars=[('A', 'D')], value_vars=[('B', 'E')])
